In [1]:
# dependencies
import os
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

In [2]:
#Setup database
app = Flask(__name__)
# app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///ffdb"
# db = SQLAlchemy(app)
ffdb = sqlite3.connect('ffdb')

# Pull data from database into a local variable
# This would be really unacceptable if our database wasn't basically a tiny little spreadsheet
# If complexity increased, it would be very reasonable to tailor select statements to specific requests inside of the functions I call below
cursor = ffdb.cursor()
cursor.execute('''
    SELECT * from fire
''')
data = cursor.fetchall()

In [3]:
# The date fixer function that, frankly, I can't even believe I needed to make
def dateFixer(month, year):
    portugueseMonths = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho',
       'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']
    n = 1
    monthDictionary = {}
    
    for monthName in portugueseMonths:
        if len(str(n)) == 1:
            monthDictionary[monthName] = "0" + str(n)
        else:
            monthDictionary[monthName] = str(n)
        n+=1
        
    date = str(year) + "-" + monthDictionary[month] + "-01"
    return date

In [4]:
# a dataframe for your enjoyment
df_data = []
for row in data:
    row = list(row)
    row[4] = dateFixer(row[2],row[0])
    if row[3] != np.floor(row[3]):
        fixed_number = row[3]*1000
        row[3] = fixed_number
    df_data.append(row)

ffdf = pd.DataFrame(df_data, columns=["year","state","month","number","date"])
ffdf.head(200)

,year,state,month,number,date
0,1998,Acre,Janeiro,0.0,1998-01-01
1,1999,Acre,Janeiro,0.0,1999-01-01
2,2000,Acre,Janeiro,0.0,2000-01-01
3,2001,Acre,Janeiro,0.0,2001-01-01
4,2002,Acre,Janeiro,0.0,2002-01-01
5,2003,Acre,Janeiro,10.0,2003-01-01
6,2004,Acre,Janeiro,0.0,2004-01-01
7,2005,Acre,Janeiro,12.0,2005-01-01
8,2006,Acre,Janeiro,4.0,2006-01-01
9,2007,Acre,Janeiro,0.0,2007-01-01


In [5]:
# Dataframe sorting
ffdf = ffdf.sort_values(by = "date")
data_sorted = []
for r in ffdf.values:
    data_sorted.append(tuple(r))

In [34]:
grouped_df = ffdf.groupby("state").sum()
ffdf_grouped = pd.DataFrame(grouped_df["number"])
ffdf_grouped = ffdf_grouped.reset_index()

ind_state_list = ffdf_grouped["state"].values.tolist()
fire_list = ffdf_grouped["number"].values.tolist()

['Acre',
 'Alagoas',
 'Amapa',
 'Amazonas',
 'Bahia',
 'Ceara',
 'Distrito Federal',
 'Espirito Santo',
 'Goias',
 'Maranhao',
 'Mato Grosso',
 'Minas Gerais',
 'Paraiba',
 'Pará',
 'Pernambuco',
 'Piau',
 'Rio',
 'Rondonia',
 'Roraima',
 'Santa Catarina',
 'Sao Paulo',
 'Sergipe',
 'Tocantins']

In [ ]:
# TO DO
# Create a flask app route that gives the necessary JSONified information for making the pie chart work
@app.route("/pie/")
def pie():
    i = 0
    pie_dict= {}
    for state in ind_state_list:
        row = {}
        row["state"] = state
        row["number"] = fire_list[i]
        pie_dict[i] = row
        i += 1
    print(pie_dict)
    return jsonify(pie_dict)

@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")


@app.route("/states")
def states():
    """Return a list of sample names."""
    
    return jsonify(['Acre', 'Alagoas', 'Amapa', 'Amazonas', 'Bahia', 'Ceara', 'Distrito Federal', 'Espirito Santo', 'Goias', 'Maranhao', 'Mato Grosso', 'Minas Gerais', 'Pará', 'Paraiba', 'Pernambuco', 'Piau', 'Rio', 'Rondonia', 'Roraima', 'Santa Catarina', 'Sao Paulo', 'Sergipe', 'Tocantins'])


@app.route("/state_data/<state>")
#  def sample_metadata(sample):
def state_data(state):
   

    """Return the Data for a given state."""
    results = []
    for x in data_sorted:
        if x[1] == state:
            results.append(x)
#     print(results)
    


    # Create a dictionary entry for each row of information
    state_data_dict = {}
    n = 0
    
    # turn each row into its own dictionary and add that to the big dictionary
    for result in results:
        state_data = {}
        state_data["year"] = result[0]
        state_data["state"] = result[1]
        state_data["month"] = result[2]
        state_data["number"] = result[3]
        state_data["date"] = dateFixer(state_data["month"], state_data["year"])
        state_data_dict[n] = state_data
        n += 1
    
#return that info jsonified
#     print(state_data_dict)
    return jsonify(state_data_dict)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
